# mRNA EDA
## for miRNA-Target Interactions in Cancer

In [1]:
import csv
import google.datalab.storage as storage
import io
import logging
import math as m
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns

## TODO: Edit and test plotting code to save to GCP bucket

## Utils

### Function definitions

In [2]:
def read_file(bucket, filepath, **kwargs):
  uri = bucket.object(filepath).uri
  get_ipython().run_line_magic('gcs', 'read --object ' + uri + ' --variable csv_data')
  return pd.read_csv(io.BytesIO(csv_data), **kwargs)

In [3]:
def save_as(temp_filename, filepath):
  !gsutil cp $temp_filename $filepath

### Set up logging

In [4]:
logger = logging.getLogger()

In [5]:
def setup_file_logger(log_file):
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)
    
def log(message):
  print(message)
  logger.info(message)

setup_file_logger('mirtar.log')

## Preprocess data

#### Read sample mRNA expression data

In [6]:
bucket = storage.Bucket('yfl-mirna')

In [7]:
mRNA_data = read_file(bucket, 'data/mRNA/EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', delimiter='\t')
mRNA_data.rename(index=str, columns={'gene_id': 'mRNA'}, inplace=True)
mRNA_data.set_index('mRNA', inplace=True)

In [8]:
mRNA_data.shape

(20531, 11069)

In [9]:
sample_mRNAs = mRNA_data.T
sample_mRNAs.index = sample_mRNAs.index.map(lambda x: '-'.join(x.split('-')[0:4]))
sample_mRNAs.reset_index(inplace=True)
sample_mRNAs.drop_duplicates(subset='index', keep='first', inplace=True)
sample_mRNAs.set_index('index', inplace=True)

#### Read sample metadata

In [10]:
sample_metadata = read_file(bucket, 'data/sample/PanCanAtlas_miRNA_sample_information_list.txt', delimiter='\t')
sample_metadata.rename(index=str, columns={'id': 'sample'}, inplace=True)
sample_metadata.set_index('sample', inplace=True)
sample_metadata.index = sample_metadata.index.map(lambda x: '-'.join(x.split('-')[0:4]))
sample_metadata.reset_index(inplace=True)
sample_metadata.drop_duplicates(subset='sample', keep='first', inplace=True)
sample_metadata.set_index('sample', inplace=True)

In [11]:
sample_metadata

,Disease,Sample_Type,Protocol,Platform
sample,,,,
TCGA-OR-A5J1-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J2-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J3-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J4-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J5-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J6-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J7-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J8-01A,ACC,1,Direct,HiSeq
TCGA-OR-A5J9-01A,ACC,1,Direct,HiSeq


#### Merging mRNA expression data and (miRNA) sample metadata

In [10]:
samples = sample_metadata.merge(sample_mRNAs, left_index=True, right_index=True)

#### Definition: Considering samples of type 1 only

In [11]:
type_1_samples = pd.DataFrame(samples[samples.Sample_Type == 1])
type_1_sample_mRNAs = type_1_samples.drop(columns=sample_metadata.columns)
type_1_mRNA_data = type_1_sample_mRNAs.T

In [12]:
type_1_samples_count = type_1_samples.shape[0]

## Very high-level overview of mRNA-miRNA data

#### Number of samples for each tumor type

In [13]:
tumor_types_and_counts = type_1_samples['Disease'].value_counts().sort_index()
tumor_types = tumor_types_and_counts.index
tumor_type_counts = tumor_types_and_counts.values
tumor_types_num = tumor_types.size

#### Bar chart

In [ ]:
fig, ax = plt.subplots()
bar_width = 5
spacing = 10
indices = np.arange(0, len(tumor_types) * spacing, spacing)
ax.bar(indices, tumor_type_counts, bar_width)
ax.set_xlabel('Tumor type', fontsize=12)
ax.set_ylabel('Number of samples', fontsize=12)
ax.set_title('Number of samples by tumor type', fontsize=14)
ax.set_xticks(indices)
ax.set_xticklabels(tumor_types, rotation=90)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(12, 7.2)
fig.savefig('mRNA-miRNA_number-of-samples_barchart.pdf')

#### Pie chart

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Samples by tumor type')
plt.pie(tumor_type_counts, labels = tumor_types, autopct='%1.1f%%')
fig.set_size_inches(20, 20)
fig.savefig('mRNA-miRNA_number-of-samples_piechart.pdf')

### Summary of mRNA type-1 sample data

#### Overall descriptive summary

In [ ]:
type_1_mRNA_data.describe()

#### Maximum sample expression value(s)

In [23]:
type_1_mRNA_data.max().max()

7092440.0

In [34]:
type_1_mRNA_data.max().describe()

count    9.310000e+03
mean     4.272301e+05
std      5.165901e+05
min      5.536860e+04
25%      1.658308e+05
50%      2.567420e+05
75%      4.722172e+05
max      7.092440e+06
dtype: float64

#### MInimum sample expression value is very slightly larger than -1

In [24]:
type_1_mRNA_data.min().min()

-0.99121057347755

In [33]:
type_1_mRNA_data.min().describe()

count    9310.000000
mean       -0.135038
std         0.311809
min        -0.991211
25%         0.000000
50%         0.000000
75%         0.000000
max         0.000000
dtype: float64

#### Sample expression medians

In [37]:
type_1_mRNA_data.median(axis=1).describe()

count    20531.000000
mean       714.378856
std       2138.330624
min          0.000000
25%          4.743725
50%        178.202110
75%        740.260000
max      94209.329512
dtype: float64

### Distribution of missing values across mRNAs and samples

In [14]:
type_1_mRNA_data_nonnull_mask = type_1_mRNA_data.isnull()
type_1_mRNA_na_counts = type_1_mRNA_data_nonnull_mask.sum(axis=1)
type_1_sample_na_counts = type_1_mRNA_data_nonnull_mask.sum()

#### Total # of missing values

In [26]:
type_1_mRNA_na_counts.sum()

3107138

#### Number of mRNAs / samples with missing values

In [27]:
len(type_1_mRNA_na_counts[type_1_mRNA_na_counts != 0])

4196

In [28]:
len(type_1_sample_na_counts[type_1_sample_na_counts != 0])

1478

#### Distribution of # of missing values across mRNAs / samples

In [29]:
type_1_mRNA_na_counts.describe()

count    20531.000000
mean       151.338853
std        393.685936
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       1478.000000
dtype: float64

In [30]:
type_1_mRNA_na_counts.value_counts()

0       16335
38       1107
1478     1050
592       869
554       376
1445      368
625       312
587        49
891        33
71         23
33          5
924         4
dtype: int64

In [31]:
type_1_sample_na_counts.describe()

count    9310.000000
mean      333.741998
std       831.914826
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      3766.000000
dtype: float64

In [32]:
type_1_sample_na_counts.value_counts()

0       7832
1455     853
3024     554
3766      38
1443      33
dtype: int64

## Plots

### Function definitions

In [147]:
def histogram(data, xsz, ysz, title, xlab, ylab, xticks, **kwargs):
    fig, ax = plt.subplots()
    sns.distplot(data, ax=ax, **kwargs)
    fig = ax.get_figure()
    fig.set_size_inches(xsz, ysz)
    ax.set_title(title)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    if xticks is not None:
      ax.set_xticks(xticks)
    return fig

In [16]:
def heatmap(data, title, xlab, ylab, xticks, yticks, xticklabels, yticklabels, xsz, ysz, cmap, **kwargs):
    fig, ax = plt.subplots()
    if 'label_fontsize' in kwargs:
        ax.set_title(title, fontsize=kwargs['label_fontsize'])
        ax.set_xlabel(xlab, fontsize=kwargs['label_fontsize'])
        ax.set_ylabel(ylab, fontsize=kwargs['label_fontsize'])
    else:
        ax.set_title(title)
        ax.set_xlabel(xlab)
        ax.set_ylabel(ylab)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    xticklab_dict = dict()
    if ('xticklabels_rotation' in kwargs):
        xticklab_dict['rotation'] = kwargs['xticklabels_rotation']
    if ('ha' in kwargs):
        xticklab_dict['ha'] = kwargs['ha']
    ax.set_xticklabels(xticklabels, xticklab_dict)
    if 'va' in kwargs:
        ax.set_yticklabels(yticklabels, va=kwargs['va'])
    else:
        ax.set_yticklabels(yticklabels)
    fig.set_size_inches(xsz, ysz)
    heatmap = ax.pcolormesh(data, cmap=cmap)
    fig.colorbar(heatmap, ax=ax)
    return fig

### Data definitions

In [17]:
mRNA_data_log_transformed = type_1_mRNA_data.applymap(lambda x: m.log(x + 2))

In [18]:
sample_disease_mRNAs = type_1_samples.drop(columns=['Sample_Type', 'Protocol', 'Platform'])
samples_by_tumor_type = sample_disease_mRNAs.groupby('Disease')

In [19]:
tumor_type_sample_counts_cumulative = np.cumsum(samples_by_tumor_type.size().values)

In [20]:
samples_sortedby_tumor_type = sample_disease_mRNAs.sort_values(by=['Disease'])
sample_mRNAs_sortedby_tumortype = samples_sortedby_tumor_type.drop(columns=['Disease'])
sample_mRNAs_sortedby_tumortype_log = sample_mRNAs_sortedby_tumortype.applymap(lambda x: m.log(x + 2))
sample_tumor_types_sorted = samples_sortedby_tumor_type['Disease']

### Missing values

In [21]:
sample_mRNAs_sortedby_tumortype_isnull = sample_mRNAs_sortedby_tumortype.isnull()
sample_mRNAs_sortedby_tumortype_isnull['Disease'] = sample_tumor_types_sorted
sample_mRNAs_isnull_groupedby_tumortype = sample_mRNAs_sortedby_tumortype_isnull.groupby('Disease')
sample_mRNAs_groupedby_tumortype_nullcounts = sample_mRNAs_isnull_groupedby_tumortype.sum()
sample_mRNAs_groupedby_tumortype_null_percentages = sample_mRNAs_groupedby_tumortype_nullcounts.div(tumor_types_and_counts, axis='index')

In [ ]:
fig = heatmap(sample_mRNAs_groupedby_tumortype_null_percentages,
              'missing mRNA expression values as % of cancer type samples', 
              'mRNA', '% cancer type samples with missing expression values for mRNA', 
              np.arange(sample_mRNAs_groupedby_tumortype_nullcounts.shape[1]), np.arange(tumor_types_num), 
              [], tumor_types, 20, 10, 'hot_r', label_fontsize=16, va='bottom')
fig.savefig('mRNA-tumortype-missing-percentages-heatmap.png')

### mRNA expression sorted by tumor type and mean expression value

In [22]:
mRNA_samples_by_tumor_type = sample_mRNAs_sortedby_tumortype.T
mRNA_samples_by_tumor_type['mean_expr'] = mRNA_samples_by_tumor_type.mean(axis=1)
mRNA_samples_by_tumor_type.sort_values(by=['mean_expr'], inplace=True)
mRNA_samples_by_tumor_type.drop(columns=['mean_expr'], inplace=True)

sample_mRNAs_by_meanexpr_and_tumortype = mRNA_samples_by_tumor_type.T
sample_mRNAs_by_meanexpr_and_tumortype['Disease'] = sample_tumor_types_sorted
sample_mRNAs_sortedby_meanexpr_groupedby_tumortype = sample_mRNAs_by_meanexpr_and_tumortype.groupby('Disease')
mRNA_tumortype_means = sample_mRNAs_sortedby_meanexpr_groupedby_tumortype.mean()

#### Mean sample expression

In [ ]:
fig = heatmap(mRNA_tumortype_means.applymap(lambda x: m.log(x + 2)),
              'mRNA log((tumor type mean sample expression) + 2), sorted by mRNA mean expression level', 
              'mRNA', 'log((tumor type mean sample expression) + 2)', 
              np.arange(len(sample_mRNAs_by_meanexpr_and_tumortype.columns[:-1])), np.arange(tumor_types_num), 
              [], tumor_types, 50, 15, 'cubehelix_r', label_fontsize=16, va='bottom')
fig.savefig('mRNA-tumortype-mean-exprs-heatmap.png')

#### Per-mRNA sample expression quantiles

In [45]:
quantiles = [i/100 for i in list(range(5, 105, 5))]
per_mRNA_quantiles = mRNA_samples_by_tumor_type.quantile(q=quantiles, axis=1)

In [ ]:
heatmap(per_mRNA_quantiles.applymap(lambda x: m.log(x + 2)),
        'mRNA log((per-mRNA expression quantile) + 2), sorted by mRNA mean expression level', 'mRNA',
        'log((expression quantile) + 2)', np.arange(len(sample_mRNAs_by_meanexpr_and_tumortype.columns[:-1])),
        range(len(quantiles)), [], list(quantiles), 50, 20, 'cubehelix_r',
        label_fontsize=16, va='bottom').savefig('mRNA-quantiles_sortedby-means_heatmap.png')

#### Per-mRNA sample expression quantiles, normalised

In [23]:
mRNA_sample_means = mRNA_samples_by_tumor_type.mean(axis=1)
mRNA_sample_stdevs = mRNA_samples_by_tumor_type.std(axis=1)

In [160]:
# compare to medians above [type_1_mRNA_data.median(axis=1).describe()]
mRNA_sample_means.describe()

count     20531.000000
mean        988.510438
std        2880.887841
min          -0.023894
25%          42.880666
50%         332.102388
75%         986.616282
max      103669.311442
dtype: float64

In [47]:
mRNA_null_counts = per_mRNA_quantiles.sub(mRNA_sample_means, axis=1).div(mRNA_sample_stdevs, axis=1).isnull().sum()
mRNA_null_counts[mRNA_null_counts != 0].describe()

count    223.0
mean      20.0
std        0.0
min       20.0
25%       20.0
50%       20.0
75%       20.0
max       20.0
dtype: float64

In [24]:
# using the full range (up to 1) blasts the maximum from around 3 to 80... (resulting in white-out everywhere but top quantile)
quantiles = [i/100 for i in list(range(5, 100, 5))]
per_mRNA_quantiles = mRNA_samples_by_tumor_type.quantile(q=quantiles, axis=1)

In [ ]:
fig = heatmap(per_mRNA_quantiles.sub(mRNA_sample_means, axis=1).div(mRNA_sample_stdevs, axis=1),
              'normalised per-mRNA expression quantile, sorted by mRNA mean expression level', 
              'mRNA', 'normalised per-mRNA expression quantile', np.arange(len(sample_mRNAs_by_meanexpr_and_tumortype.columns[:-1])),
              range(len(quantiles)), [], list(quantiles), 50, 20, 'cubehelix_r', label_fontsize=16, va='bottom')
fig.savefig('mRNA-normed-quantiles_sortedby-means_heatmap.png')

### mRNA expression sorted by tumor type and variance in per-cancer-type mean expression

In [25]:
mRNA_samples_by_tumor_type = sample_mRNAs_sortedby_tumortype.T
mRNA_samples_by_tumor_type['stdev_of_tumor_type_mean'] = samples_by_tumor_type.mean().std().T
mRNA_samples_by_tumor_type.sort_values(by=['stdev_of_tumor_type_mean'], inplace=True)
mRNA_samples_by_tumor_type.drop(columns=['stdev_of_tumor_type_mean'], inplace=True)

sample_mRNAs_by_meanstdev_and_tumortype = mRNA_samples_by_tumor_type.T
sample_mRNAs_by_meanstdev_and_tumortype['Disease'] = sample_tumor_types_sorted
sample_mRNAs_sortedby_meanstdev_groupedby_tumortype = sample_mRNAs_by_meanstdev_and_tumortype.groupby('Disease')
mRNA_tumortype_means = sample_mRNAs_sortedby_meanstdev_groupedby_tumortype.mean()

#### Mean sample expression

In [ ]:
fig = heatmap(mRNA_tumortype_means.applymap(lambda x: m.log(x + 2)),
              'Heatmap: mRNA log((tumor type mean sample expression) + 2), sorted by variance in per-cancer-type means', 
              'mRNA', 'log((tumor type mean sample expression) + 2)', 
              np.arange(len(sample_mRNAs_by_meanstdev_and_tumortype.columns[:-1])), np.arange(tumor_types_num), 
              [], tumor_types, 50, 15, 'cubehelix_r', label_fontsize=16, va='bottom')
fig.savefig('mRNA-tumortype-mean-exprs-heatmap_sortedby-meanstdevs.png')

### Excluded due to excessive runtime:
- Boxplots of raw sample expression values and summary stats (mean, quantiles) for each mRNA
- Heatmap of pairwise mRNA correlations (correlation computation takes too long, even within cancer types)
- Heatmaps of raw and normalised per-sample expression values

#### Grouped boxplot of sample expression for mRNAs with highest and lowest variation in mean expression across tumor types

In [26]:
quantiles = [i/100 for i in list(range(5, 105, 5))]
per_mRNA_quantiles = mRNA_samples_by_tumor_type.quantile(q=quantiles, axis=1)
miRNA_null_counts = per_mRNA_quantiles.sub(mRNA_sample_means, axis=1).div(mRNA_sample_stdevs, axis=1).isnull().sum()
mRNA_samples_by_tumor_type_no_0stdevs = mRNA_samples_by_tumor_type.drop(miRNA_null_counts[miRNA_null_counts != 0].index)

In [27]:
most_varied_mRNA = mRNA_samples_by_tumor_type_no_0stdevs.iloc[-1].name
least_varied_mRNA = mRNA_samples_by_tumor_type_no_0stdevs.iloc[0].name

In [ ]:
most_least_varied_mRNAs = pd.concat([sample_mRNAs_sortedby_tumortype_log[most_varied_mRNA],
                                     sample_mRNAs_sortedby_tumortype_log[least_varied_mRNA],
                                     samples_sortedby_tumor_type['Disease']], axis=1)
ax1, ax2 = most_least_varied_mRNAs.boxplot(by='Disease', whis='range', showfliers=False, figsize=(10, 7))
plt.suptitle('log((tumor type sample expression) + 2) for mRNAs with highest and lowest nonzero variation across tumor types')
ax1.set_xlabel('Tumor Type')
ax2.set_xlabel('Tumor Type')
ax1.set_ylabel('log((tumor type sample expression) + 2)')
xticks = range(1, tumor_types_num + 2) # +2 because pandas is weird
ax1.set_xticks(xticks)
ax2.set_xticks(xticks)
xticklabels = [''] + tumor_types.tolist()
ax1.set_xticklabels(xticklabels, rotation=90, ha='right')
ax2.set_xticklabels(xticklabels, rotation=90, ha='right')
plt.savefig('most-least-varied-mRNAs-expr-by-tumortype_boxplot.png')

### Observations

##### Data characteristics
- 20531 mRNAs, 9310 (type 1, primary solid tumor) samples
- Minimum sample expression value is -0.99121057347755 (very slightly larger than -1)
- Max. sample expression value: 7092440
- Missing values:
  - 3107138 (~16%) in dataset
  - 4196 (~20%) mRNAs with missing expression values
    - mean 151.338853, stdev 393.685936, max 1478
    
#### Per-cancer-type mean expression
- Distribution of log-transformed expression (+2) seems evenly distributed
  - Though with slightly more values at bottom than top end of range
- Varies primarily across mRNAs rather than across cancer types
  - Fairly even across cancer types for most mRNAs
- Shows a more striated pattern sorted by variance of per-cancer-type means than sorted by overall expression mean 
  - I.e. some mRNAs show large variation away from the top of the distribution

#### Per-mRNA sample expression quantiles (sorted by mRNA overall mean expression)
- Raw
  - Stark contrast between top quantile (max) values and lower ones
  - Below the top quantile, most mRNA expression quantiles cover relatively small range
- Normalised per mRNA
  - Capped at 95th percentile:
    - Lowest-mean distributions are quite uniform
    - Dispersion increases with mean
      - Though again there is a fatter right tail than left tail
    - For the top quantiles, a sharp contrast with the next quantile displayed
      - Starting farther to the right as the quantile decreases
        - I.e. starting at higher-mean mRNAs for lower (but still high) quantiles
    - Range: > -3 to -3
  - Up to 100th percentile:
    - Value of top quantile (maximum) is blasted up to 80, while all other quantile languish close to 0
    - However, the (normalised) maximum value is highest for lowest-mean mRNAs, and decreases with increasing mean
    
#### mRNAs with highest and lowest variation in mean expression across tumor types
- Zero variation is recorded for some mRNAs (with only 0 values recorded)
- Some others have zero variation (and 0 values) in most cancer types

## Analysis

In [28]:
nranks = 25

### Ranking of mRNAs by expression levels

#### By mean over all samples

In [52]:
# last column of sample_mRNAs_by_meanexpr_and_tumortype is 'Disease'
top25_mRNAs_by_overall_mean = sample_mRNAs_by_meanexpr_and_tumortype.columns[-2:(-2-nranks):-1]
top25_mRNAs_by_overall_mean_idx = {}
for i in range(nranks):
    top25_mRNAs_by_overall_mean_idx[top25_mRNAs_by_overall_mean[i]] = i

#### By mean of per-tumor-type means

In [53]:
mRNA_tumortype_means_sortedby_mean_of_means = mRNA_tumortype_means.mean().sort_values()

In [54]:
top25_mRNAs_by_mean_of_means = mRNA_tumortype_means_sortedby_mean_of_means.index[-1:(-1-nranks):-1]
top25_mRNAs_by_mean_of_means_idx = {}
for i in range(nranks):
    top25_mRNAs_by_mean_of_means_idx[top25_mRNAs_by_mean_of_means[i]] = i

#### Size of intersection

In [55]:
len(top25_mRNAs_by_overall_mean.intersection(top25_mRNAs_by_mean_of_means))

22

In [56]:
top25_mRNAs_by_overall_mean[0:3]

Index(['ACTB|60', 'EEF1A1|1915', 'GAPDH|2597'], dtype='object')

In [57]:
top25_mRNAs_by_mean_of_means[0:3]

Index(['ACTB|60', 'EEF1A1|1915', 'GAPDH|2597'], dtype='object')

#### Mean difference between mRNA ranks
- Counting only mRNAs in intersection (i.e. in both)
- Subtracting rank by mean of means from rank by overall means

In [58]:
sum_of_diffs = 0
count_of_diffs = 0
for i in range(nranks):
    if top25_mRNAs_by_overall_mean[i] in top25_mRNAs_by_mean_of_means_idx:
        sum_of_diffs += i - top25_mRNAs_by_mean_of_means_idx[top25_mRNAs_by_overall_mean[i]]
        count_of_diffs += 1
sum_of_diffs / count_of_diffs

-0.18181818181818182

#### Mean absolute difference between mRNA ranks

In [59]:
sum_of_abs_diffs = 0
count_of_diffs = 0
for i in range(nranks):
    if top25_mRNAs_by_overall_mean[i] in top25_mRNAs_by_mean_of_means_idx:
        sum_of_abs_diffs += abs(i - top25_mRNAs_by_mean_of_means_idx[top25_mRNAs_by_overall_mean[i]])
        count_of_diffs += 1
sum_of_abs_diffs / count_of_diffs

1.9090909090909092

#### Differences in differences between the rankings

In [60]:
mRNAs_by_overall_mean = sample_mRNAs_by_meanexpr_and_tumortype.columns[::-1][1:]
mRNAs_by_mean_of_means = mRNA_tumortype_means_sortedby_mean_of_means.index[::-1]

#### Mean difference between ranks of mRNAs in differences
- The mean absolute differences would have the same magnitudes

#### mRNAs in top 25 by overall mean but not by mean of means

In [61]:
sum_of_diffs = 0
count_of_diffs = 0
for i in range(nranks):
    if not(top25_mRNAs_by_overall_mean[i] in top25_mRNAs_by_mean_of_means_idx):
        sum_of_diffs += i - mRNAs_by_mean_of_means.get_loc(top25_mRNAs_by_overall_mean[i])
        count_of_diffs += 1
sum_of_diffs / count_of_diffs

-23.666666666666668

#### mRNAs in top 25 by mean of means but not by overall mean

In [62]:
sum_of_diffs = 0
count_of_diffs = 0
for i in range(nranks):
    if not(top25_mRNAs_by_mean_of_means[i] in top25_mRNAs_by_overall_mean_idx):
        sum_of_diffs += i - mRNAs_by_overall_mean.get_loc(top25_mRNAs_by_mean_of_means[i])
        count_of_diffs += 1
sum_of_diffs / count_of_diffs

-16.333333333333332

#### Boxplot of sample expression for mRNAs with highest and lowest mean expressions across all samples

In [ ]:
fig, ax = plt.subplots()
most_least_varied_mRNAs = pd.concat([sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[-2]],
                                     sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[-3]],
                                     sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[-4]],
                                     sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[0]],
                                     sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[1]],
                                     sample_mRNAs_sortedby_tumortype_log[sample_mRNAs_by_meanexpr_and_tumortype.columns[2]],
                                     samples_sortedby_tumor_type['Disease']], axis=1)
most_least_varied_mRNAs.boxplot(ax=ax, whis='range', showfliers=False)
title = 'log((sample expression) + 2) for mRNAs with highest and lowest expression levels, ranked by mean over dataset'
ax.set_title(title, fontsize=14)
ax.set_xlabel('mRNA', fontsize=12)
ax.set_ylabel('log((tumor type sample expression) + 2)', fontsize=12)
fig.set_size_inches(12, 8.4)
fig.savefig('mRNAs-highest-lowest-mean-expr_boxplot.png')

#### Boxplot of sample expression for mRNAs with highest and lowest means of per-tumor-type mean expressions

In [ ]:
fig, ax = plt.subplots()
most_least_varied_mRNAs = pd.concat([sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[-2]],
                                     sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[-3]],
                                     sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[-4]],
                                     sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[0]],
                                     sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[1]],
                                     sample_mRNAs_by_meanexpr_and_tumortype[mRNA_tumortype_means_sortedby_mean_of_means.index[2]],
                                     samples_sortedby_tumor_type['Disease']], axis=1)
most_least_varied_mRNAs.boxplot(ax=ax, whis='range', showfliers=False)
ax.set_title('log((sample expression) + 2) for mRNAs with highest and lowest expression levels, ranked by mean of per-tumor-type means', fontsize=12)
ax.set_xlabel('mRNA', fontsize=12)
ax.set_ylabel('log((tumor type sample expression) + 2)', fontsize=12)
fig.set_size_inches(12, 8.4)
fig.savefig('mRNAs-highest-lowest-mean-of-means-expr_boxplot.png')

#### Observations:
- Generally, difference between the two rankings reflects a difference between degree of variation across tumor types and degree of variation across samples (the entire dataset)
- The two rankings are similar but not identical:
  - Out of the 25 in each ranking, 22 mRNAs are in both
  - The top 3 in both are identical: ACTB|60, EEF1A1|1915, GAPDH|2597
  - The mean difference in position between mRNAs in both is small, at -0.1818
  - The mean absolute difference is slightly higher, at 1.9091
  - This suggests a very slight concentration overall of high values in tumor types with larger sample sizes relative to the distribution/concentration over all samples
  - For the 3 in the top 25 by overall mean but not by mean of means:
    - The mean difference is -23.67, suggesting a higher concentration of high values in large-sample cancer types
      - The mean absolute difference has same magnitude as the mean difference
  - For the 3 in the top 25 by mean of means but not by overall mean:
    - The mean difference is -16.33, suggesting a somewhat high concentration of high values in small-sample cancer types, though less imbalanced than for the corresponding large-sample high-value concentration for mRNAs only in the top 25 by overall mean
      - Similarly, the mean absolute difference has same magnitude as the mean difference

#### Questions
- How will these results change when restricting the analysis to different sets of cancer types?
- How are the two rankings correlated?
- What's the formal/mathematical relationship between the mean over all samples, and the mean of means over subsets of the dataset?

### Ranking of mRNAs by variation across tumor types

#### Using the mean expression of each mRNA per cancer type, and then computing the variance

In [63]:
# At time of writing, mRNA_samples_by_tumor_type should be appropriately sorted at this point in the notebook,
# but just in case...
mRNA_samples_by_tumor_type['stdev_of_tumor_type_mean'] = samples_by_tumor_type.mean().std().T
mRNA_samples_by_tumor_type.sort_values(by=['stdev_of_tumor_type_mean'], inplace=True)
mRNA_samples_by_tumor_type.drop(columns=['stdev_of_tumor_type_mean'], inplace=True)
mRNA_samples_sortedby_stdev_of_tumortype_means = mRNA_samples_by_tumor_type
top25_mRNAs_by_stdev_of_means = mRNA_samples_by_tumor_type.index[-1:(-1-nranks):-1]

In [64]:
bottom25_mRNAs_by_stdev_of_means = mRNA_samples_by_tumor_type.index[:nranks]

#### Using a one-way ANOVA test and ranking the mRNAs based on their P-values
- Most mRNAs have 0 p-values, so I also sorted by F-statistic value: there are still many 0-p-value mRNAs not included

In [65]:
mRNA_samples_by_tumor_type = samples_by_tumor_type.obj.T
mRNA_samples_by_tumor_type.drop(['Disease'], inplace=True)
mRNA_1was = mRNA_samples_by_tumor_type.apply(lambda x: stats.f_oneway(*samples_by_tumor_type[x.name].apply(list)), axis=1)
mRNA_samples_by_tumor_type['1way_anova_fstat'] = mRNA_1was.map(lambda x: x.statistic)
mRNA_samples_by_tumor_type['1way_anova_pval'] = mRNA_1was.map(lambda x: x.pvalue)

In [67]:
mRNA_samples_by_tumor_type_groupedby_1wa_pval = mRNA_samples_by_tumor_type.groupby(['1way_anova_pval'])
mRNA_samples_groupedby_1wa_pvals = mRNA_samples_by_tumor_type_groupedby_1wa_pval.apply(lambda x: x.sort_values(by=['1way_anova_fstat'], ascending=False))
top25_mRNAs_by_1wa_pvals = mRNA_samples_groupedby_1wa_pvals.loc[0].iloc[0:nranks].index

In [126]:
bottom25_mRNAs_by_1wa_pvals = mRNA_samples_groupedby_1wa_pvals.iloc[-1:(-1-nranks):-1].apply(lambda mRNA: mRNA.name[1], axis=1).values

#### Using a Kruskal-Wallis test and ranking the mRNAs based on their P-values
- Most mRNAs have 0 p-values, so I also sorted by F-statistic value: there are still many 0 p-values not included

In [128]:
mRNA_samples_by_tumor_type = samples_by_tumor_type.obj.T
mRNA_samples_by_tumor_type.drop(['Disease'], inplace=True)
mRNA_kws = mRNA_samples_by_tumor_type.apply(lambda x: stats.kruskal(*samples_by_tumor_type[x.name].apply(list)), axis=1)
mRNA_samples_by_tumor_type['kruskal_wallis_hstat'] = mRNA_kws.map(lambda x: x.statistic)
mRNA_samples_by_tumor_type['kruskal_wallis_pval'] = mRNA_kws.map(lambda x: x.pvalue)

In [129]:
mRNA_samples_by_tumor_type_groupedby_kw_pval = mRNA_samples_by_tumor_type.groupby(['kruskal_wallis_pval'])
mRNA_samples_groupedby_kw_pvals = mRNA_samples_by_tumor_type_groupedby_kw_pval.apply(lambda x: x.sort_values(by=['kruskal_wallis_hstat'], ascending=False))
top25_mRNAs_by_kw_pvals = mRNA_samples_groupedby_kw_pvals.loc[0].iloc[0:nranks].index

In [130]:
bottom25_mRNAs_by_kw_pvals = mRNA_samples_groupedby_kw_pvals.iloc[-1:(-1-nranks):-1].apply(lambda mRNA: mRNA.name[1], axis=1).values

#### Intersections and their sizes

In [133]:
top25_mRNAs_mean_stdevs_1wa_intersection = top25_mRNAs_by_stdev_of_means.intersection(top25_mRNAs_by_1wa_pvals)
top25_mRNAs_mean_stdevs_kw_intersection = top25_mRNAs_by_stdev_of_means.intersection(top25_mRNAs_by_kw_pvals)
top25_mRNAs_1wa_kw_intersection = pd.Index(list(set(top25_mRNAs_by_1wa_pvals).intersection(set(top25_mRNAs_by_kw_pvals))))
top25_mRNAs_3way_intersection = top25_mRNAs_by_stdev_of_means.intersection(top25_mRNAs_1wa_kw_intersection)

In [134]:
print(top25_mRNAs_mean_stdevs_1wa_intersection)
print(top25_mRNAs_mean_stdevs_kw_intersection)
print(top25_mRNAs_1wa_kw_intersection)
print(top25_mRNAs_3way_intersection)

Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')
Index([], dtype='object')


In [135]:
print(len(top25_mRNAs_mean_stdevs_1wa_intersection))
print(len(top25_mRNAs_mean_stdevs_kw_intersection))
print(len(top25_mRNAs_1wa_kw_intersection))
print(len(top25_mRNAs_3way_intersection))

0
0
0
0


#### Grouped boxplot of per-cancer-type expression for mRNAs with highest and lowest variation in mean expression across tumor types, ranked by stdev of per-cancer-type means

In [136]:
mRNAs_of_interest = [mRNA_samples_sortedby_stdev_of_tumortype_means.iloc[i].name for i in [-1, -2, -3, 2, 1, 0]]

In [137]:
exprs_for_boxplot_len = len(samples_sortedby_tumor_type.index) * 6
exprs_for_boxplot_idx = pd.Index(range(exprs_for_boxplot_len))

In [138]:
tumortype_col = pd.concat([samples_sortedby_tumor_type['Disease']] * 6)
tumortype_col.index = exprs_for_boxplot_idx

In [139]:
mRNA_col = pd.Series(mRNAs_of_interest).repeat([len(samples_sortedby_tumor_type.index)] * 6)
mRNA_col.index = exprs_for_boxplot_idx

In [140]:
expr_col = pd.concat([sample_mRNAs_sortedby_tumortype_log[mRNA] for mRNA in mRNAs_of_interest])
expr_col.index = exprs_for_boxplot_idx

In [141]:
mRNA_exprs_long_for_boxplot = pd.concat([tumortype_col, mRNA_col, expr_col], axis=1)
mRNA_exprs_long_for_boxplot.rename(columns={ 0: 'mRNA', 1: 'expr' }, inplace=True)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x='Disease', y='expr', hue='mRNA', data=mRNA_exprs_long_for_boxplot, ax=ax)
fig = ax.get_figure()
fig.set_size_inches(40, 40)
ax.set_title('log((sample expression) + 2) for mRNAs with highest and lowest expression variation across cancer types, ranked by stdev of per-tumor-type means')
ax.set_xlabel('Cancer Type')
ax.set_ylabel('log((sample expression) + 2)')
fig.savefig('highest-lowest-tumortype-expr-variation_mRNA-expr_by-stdev-of-means_boxplot.png')

#### Grouped boxplot of per-cancer-type expression for mRNAs with highest and lowest variation in mean expression across tumor types, ranked by one-way Anova p-values

In [143]:
mRNAs_of_interest = [mRNA_samples_groupedby_1wa_pvals.iloc[i].name[1] for i in [0, 1, 2, -3, -2, -1]]

In [146]:
exprs_for_boxplot_len = len(samples_sortedby_tumor_type.index) * 6
exprs_for_boxplot_idx = pd.Index(range(exprs_for_boxplot_len))

In [147]:
tumortype_col = pd.concat([samples_sortedby_tumor_type['Disease']] * 6)
tumortype_col.index = exprs_for_boxplot_idx

In [148]:
mRNA_col = pd.Series(mRNAs_of_interest).repeat([len(samples_sortedby_tumor_type.index)] * 6)
mRNA_col.index = exprs_for_boxplot_idx

In [149]:
expr_col = pd.concat([sample_mRNAs_sortedby_tumortype_log[mRNA] for mRNA in mRNAs_of_interest])
expr_col.index = exprs_for_boxplot_idx

In [150]:
mRNA_exprs_long_for_boxplot = pd.concat([tumortype_col, mRNA_col, expr_col], axis=1)
mRNA_exprs_long_for_boxplot.rename(columns={ 0: 'mRNA', 1: 'expr' }, inplace=True)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x='Disease', y='expr', hue='mRNA', data=mRNA_exprs_long_for_boxplot, ax=ax)
fig = ax.get_figure()
fig.set_size_inches(40, 40)
ax.set_title('log((sample expression) + 2) for mRNAs with highest and lowest expression variation across cancer types, ranked by one-way ANOVA p-values')
ax.set_xlabel('Cancer Type')
ax.set_ylabel('log((sample expression) + 2)')
fig.savefig('highest-lowest-tumortype-expr-variation_mRNA-expr_by-1wa-pvals_boxplot.png')

#### Grouped boxplot of per-cancer-type expression for mRNAs with highest and lowest variation in mean expression across tumor types, ranked by Kruskal-Wallis p-values

In [151]:
mRNAs_of_interest = [mRNA_samples_groupedby_kw_pvals.iloc[i].name[1] for i in [0, 1, 2, -3, -2, -1]]

In [152]:
exprs_for_boxplot_len = len(samples_sortedby_tumor_type.index) * 6
exprs_for_boxplot_idx = pd.Index(range(exprs_for_boxplot_len))

In [153]:
tumortype_col = pd.concat([samples_sortedby_tumor_type['Disease']] * 6)
tumortype_col.index = exprs_for_boxplot_idx

In [154]:
mRNA_col = pd.Series(mRNAs_of_interest).repeat([len(samples_sortedby_tumor_type.index)] * 6)
mRNA_col.index = exprs_for_boxplot_idx

In [155]:
expr_col = pd.concat([sample_mRNAs_sortedby_tumortype_log[mRNA] for mRNA in mRNAs_of_interest])
expr_col.index = exprs_for_boxplot_idx

In [156]:
mRNA_exprs_long_for_boxplot = pd.concat([tumortype_col, mRNA_col, expr_col], axis=1)
mRNA_exprs_long_for_boxplot.rename(columns={ 0: 'mRNA', 1: 'expr' }, inplace=True)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(x='Disease', y='expr', hue='mRNA', data=mRNA_exprs_long_for_boxplot, ax=ax)
fig = ax.get_figure()
fig.set_size_inches(40, 40)
ax.set_title('log((sample expression) + 2) for mRNAs with highest and lowest expression variation across cancer types, ranked by Kruskal-Wallis test p-values')
ax.set_xlabel('Cancer Type')
ax.set_ylabel('log((sample expression) + 2)')
fig.savefig('highest-lowest-tumortype-expr-variation_mRNA-expr_by-kw-pvals_boxplot.png')

#### Observations
- None of the rankings intersect: the top 25 mRNAs are different in all
- The bottom 3 mRNAs are also different in all of them
- Ranking by stdev of per-cancer-type means:
  - Top 3:
    - Most per-cancer-type distributions fall within relatively low and narrow range (log-transformed expression 2 to 4)
    - Each is dominated by 1 or 2 cancer types with distributions far outside that range, or a slightly larger # of cancer types with distributions closer to but still outside or on the extremes of that range
  - Bottom 3:
    - Distributions appear flat at 0 in grouped boxplot with top 3 mRNAs
- Ranking by 1-way ANOVA p-values:
  - Top 3:
    - Most per-cancer-type distributions are more dispersed (relative to top 3 in ranking by stdev of per-cancer-type means) across wider range
    - Again, each has 1 or 2 cancer types with distributions far outside that range
  - Bottom 3:
    - Per-cancer-type distributions across a relatively but not imperceptibly narrow range—narrow relative to within-group variance?
      - Why??? Is (between-group variance / within-group variance) higher for the latter, or is there some other reason, e.g. (hard to tell from plot) within-group variance is too small, etc. (e.g. # obs / missing values)?
    - Not composed of seemingly zero-expressed mRNAs
- Ranking by Kruskal-Wallis p-values:
  - Intuitively visible how the top 3 and bottom 3 have low/high p-values under K-W (testing equality of group mean ranks), but unclear, for instance, why they don't also rank similarly under 1-way ANOVA

### mRNA expression histograms

In [ ]:
mRNA_expression_values_log = sample_mRNAs_sortedby_tumortype_log.values.flatten()
fig, ax = plt.subplots()
sns.distplot(mRNA_expression_values_log[~np.isnan(mRNA_expression_values_log)], ax=ax)
fig = ax.get_figure()
fig.set_size_inches(8, 4)
ax.set_title('Histogram: log((expression) + 2) for mRNAs across all samples')
ax.set_xlabel('log((sample expression) + 2)')
fig.savefig('log-expression-all-histogram.png')

In [157]:
sample_mRNAs_sortedby_tumortype_log['Disease'] = sample_tumor_types_sorted
sample_mRNAs_log_groupedby_tumortype = sample_mRNAs_sortedby_tumortype_log.groupby('Disease')

In [ ]:
for cancer_type, type_samples in sample_mRNAs_log_groupedby_tumortype:
    mRNA_expression_values_log = type_samples.drop(columns=['Disease']).values.flatten()
    fig, ax = plt.subplots()
    sns.distplot(mRNA_expression_values_log[~np.isnan(mRNA_expression_values_log)], ax=ax)
    fig = ax.get_figure()
    fig.set_size_inches(8, 4)
    ax.set_title('Histogram: log((expression) + 2) for mRNAs in ' + cancer_type + ' samples')
    ax.set_xlabel('log((sample expression) + 2)')
    fig.savefig('log-expression-' + cancer_type + '-histogram.png')

### Identify mRNAs with expression and variation exceeding a certain level
- using one or more metrics:
    - coefficient of variation
    - min. # samples with expression value >= 6 (or other number)
- Histogram/heatmap of number of miRNAs as parameter levels are varied / held constant
    - Histogram of # of samples (per mRNA) with expression value >= m, with m being e.g. 4 or above
        - Set m and min. # samples or percentile (requiring >= m) accordingly
    - mRNA-expression quantile heatmap?
    - mRNA-nth quantile heatmap? Or
    - mRNA-# samples (for chosen m) heatmap?

#### Coefficients of variation

In [156]:
mRNA_meta = pd.DataFrame(None, mRNA_data.index, ['coeff_of_variation'])

In [157]:
mRNA_meta['coeff_of_variation'] = stats.variation(sample_mRNAs_sortedby_tumortype.T.values, axis=1, nan_policy='omit')

In [158]:
mRNA_meta.coeff_of_variation[mRNA_meta.coeff_of_variation == 0].size

223

In [159]:
mRNA_meta.coeff_of_variation[mRNA_meta.coeff_of_variation < 0].size

2

#### Histogram: coefficient of variation in mRNA expression

In [ ]:
xaxis_max = 5 * m.ceil(mRNA_meta.coeff_of_variation.max() / 5)
xaxis_min = 5 * m.ceil(mRNA_meta.coeff_of_variation.min() / 5)
histogram(mRNA_meta.coeff_of_variation[~np.isnan(mRNA_meta.coeff_of_variation)], 20, 10, 'Histogram: mRNA coefficient of variation',
          'coefficient of variation', '# mRNAs', np.arange(xaxis_min, xaxis_max, 5), kde=False, bins=np.arange(xaxis_min, xaxis_max, 0.5)).savefig('temp.png')
save_as('temp.png', 'gs://yfl-mirna/explore/mRNA/plots/mRNA-coeff-of-var_histogram.png')

#### Histogram: # of samples (per mRNA) with expression value >= m, m in [4, 6]

In [161]:
binwidth = 500

In [ ]:
for i in range(4, 7):
  colname = 'samples_gte' + str(i) + '_count'
  mRNA_meta[colname] = type_1_sample_mRNAs[type_1_sample_mRNAs >= i].count()
  sample_count_max = binwidth * m.ceil(mRNA_meta[colname].max() / binwidth)
  histogram(mRNA_meta[colname], 10, 10, '# samples per mRNA with expression value >= ' + str(i), '# samples', '# mRNAs', None,
            kde=False, bins=range(0, sample_count_max + 1, binwidth)).savefig('temp.png')
  save_as('temp.png', 'gs://yfl-mirna/explore/mRNA/plots/mRNA-samples-gte' + str(i) + '_histogram.png')

#### Heatmap: coefficient of variation vs. # samples with expression value >= 6

In [163]:
mRNA_meta['samples_gte6_count_sup'] =  mRNA_meta.samples_gte6_count.apply(lambda x: binwidth * m.ceil(x / binwidth))
mRNA_meta['coeff_of_variation_sup'] = mRNA_meta.coeff_of_variation.apply(lambda x: m.ceil(x))
mRNA_meta_crosstab = pd.crosstab(mRNA_meta.samples_gte6_count_sup, mRNA_meta.coeff_of_variation_sup)

In [169]:
mRNA_meta['coeff_of_variation_sup'].value_counts().size

82

In [170]:
coeff_of_var_max = mRNA_meta['coeff_of_variation_sup'].max()

In [171]:
coeff_of_var_min = mRNA_meta['coeff_of_variation_sup'].min()

In [ ]:
mRNA_meta_crosstab_for_heatmap = pd.DataFrame(mRNA_meta_crosstab, mRNA_meta_crosstab.index, range(mRNA_meta_crosstab.columns.min(), mRNA_meta_crosstab.columns.max() + 1))
for i in range(mRNA_meta_crosstab.columns.min(), mRNA_meta_crosstab.columns.max() + 1):
  if i not in mRNA_meta_crosstab.columns:
    mRNA_meta_crosstab_for_heatmap[i] = 0
mRNA_meta_crosstab_for_heatmap.reindex(sorted(mRNA_meta_crosstab_for_heatmap.columns), axis=1)

In [119]:
mRNA_meta_crosstab_for_heatmap.columns

RangeIndex(start=-34, stop=90, step=1)

In [123]:
mRNA_meta_crosstab_for_heatmap.columns

RangeIndex(start=-34, stop=90, step=1)

In [ ]:
fig = heatmap(mRNA_meta_crosstab_for_heatmap.applymap(lambda x: 1000 if x > 1000 else x), '# mRNAs (capped at 1000) with: # samples with value >= 6 vs. coefficient of variation', 
              'coefficient of variation', '# samples with value >= 6', range(coeff_of_var_max - coeff_of_var_min + 1), range(1, mRNA_meta_crosstab.index.size + 1),
              mRNA_meta_crosstab_for_heatmap.columns, mRNA_meta_crosstab.index, 50, 10, 'cubehelix_r')
fig.savefig('temp.png')
save_as('temp.png', 'gs://yfl-mirna/explore/mRNA/plots/mRNA-samples-gte6_vs_coeff-of-var_heatmap.png')

### Pie-in-the-sky to-dos:
- Pairwise t-tests and Tukey's tests

### Test

In [127]:
x = [1, 2, 3]
y = [4, 5, 6]
list(zip(x, y))

[(1, 4), (2, 5), (3, 6)]

In [182]:
np.random.seed(1234321)
x2n=np.random.randn(500,2)
y2n=np.random.randn(500,2)

In [183]:
for idxs in list(zip(np.random.randint(300, size=12), np.random.randint(2, size=12))):
    x2n[idxs[0]][idxs[1]] = np.nan

In [184]:
for idxs in list(zip(np.random.randint(300, size=12), np.random.randint(2, size=12))):
    y2n[idxs[0]][idxs[1]] = np.nan

In [186]:
x2n_ma = np.ma.array(x2n, mask=np.isnan(x2n))
y2n_ma = np.ma.array(y2n, mask=np.isnan(y2n))

In [202]:
test = np.random.randn(5, 2)

In [204]:
test[0][1] = np.nan
test[3][0] = np.nan

In [211]:
test

array([[-1.54597768,         nan],
       [-0.11503133,  0.61305353],
       [-1.19194079,  0.82848454],
       [        nan,  0.84074474],
       [ 1.39147797, -1.8660572 ]])

In [212]:
~np.isnan(test)

array([[ True, False],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True]])

In [252]:
df = pd.DataFrame([[1, 0.3], [0.5, np.nan]])
# ma = pd.DataFrame([[1, 1], [1, 1]])
# ma.mask(df < 1, np.nan)

In [253]:
df < 1

,0,1
0,False,True
1,True,False


In [247]:
~df.T.isna()

,0,1
0,True,True
1,False,False


In [187]:
stats.mstats.spearmanr(x2n_ma, y2n_ma)

SpearmanrResult(correlation=0.00813648338003592, pvalue=masked_array(data=0.79959574,
             mask=False,
       fill_value=1e+20))

In [227]:
test1 = np.array([1, np.nan, np.nan])
test1_ma = np.ma.array(test1, mask=np.isnan(test1))
test2 = np.array([np.nan, 1, 1])
test2_ma = np.ma.array(test2, mask=np.isnan(test2))

In [229]:
stats.mstats.spearmanr(test1_ma, test2_ma)

ValueError: The input must have at least 3 entries!

In [231]:
test1 = pd.Series([1, np.nan, np.nan])
test2 = pd.Series([np.nan, 1, 1])
test1.corr(test2)

nan

In [ ]:
mRNA_expressions = type_1_sample_mirtars.loc[:, sample_mRNAs.columns].values.T
miRNA_expressions = type_1_sample_mirtars.loc[:, sample_miRNAs.columns].values.T
mRNA_expressions_masked = np.ma.array(mRNA_expressions, mask=np.isnan(mRNA_expressions))
miRNA_expressions_masked = np.ma.array(miRNA_expressions, mask=np.isnan(miRNA_expressions))

In [ ]:
mirtar_corrs = np.ma.corrcoef(mRNA_expressions_masked, miRNA_expressions_masked, allow_masked=True)